# Data Analysis with PostgreSQL, psycopg2, and JupyterLab
**Sally, Yoji**

## Our Dataset

### Information

- [2023 Global Country Development & Prosperity Index](https://www.kaggle.com/datasets/tarktunataalt/2023-global-country-development-and-prosperity-index)
  - Provied by kaggle.com
  - Licence: CC BY-SA 4.0
- [World population growth rate by cities 2024](https://www.kaggle.com/datasets/dataanalyst001/world-population-growth-rate-by-cities-2024/data)
  - Provied by kaggle.com
  - Licence: Apache 2.0

### Overview

#### 2023 Global Country Development & Prosperity Index

This dataset include scores which express country development such as health, economy level, governance, and so on. This dataset icludes principal countries.

#### World population growth rate by cities 2024

This dataset include population of 2023 and 2024 for each principal cities. And also, it has growth rate which is calcurated by population increase.

### Dataset detail

#### 2023 Global Country Development & Prosperity Index

##### Information

- Provied as ".csv"
- Rows: 165

##### Columns

| Coulmn name | Type | description |
| - | - | - |
| Country | varchar | The name of the country |
| AveragScore | deciaml | The overall average score of the country across all indicators |
| SafetySecurity | deciaml | Freedom from conflict, terrorism, and crime |
| PersonelFreedom | deciaml | Rights to speech, assembly, and individual autonomy |
| Governance | deciaml | Quality of democracy, rule of law, and government effectiveness |
| SocialCapital | deciaml | Strength of personal relationships and civic engagement |
| InvestmentEnvironment | deciaml | Conditions for private investment and credit access |
| EnterpriseConditions | deciaml | Business environment and market competition |
| MarketAccessInfrastructure | deciaml | Ease of trade and quality of infrastructure |
| EconomicQuality | deciaml | Macroeconomic stability and employment quality |

#### World population growth rate by cities 2024

##### Information

- Provied as ".csv"
- Rows: 801

##### Columns

| Coulmn name | Type | description |
| - | - | - |
| City | varchar | Name of the city. |
| Country | varchar | Name of the country in which the that city located in. |
| Continent | varchar | Name of the continent in which the that city located in. |
| Population (2024) | integer | Population 2024 |
| Population (2023) | integer | Population 2023 |
| Growth Rate | decimal | Growth rate of the city. |

## Our expectation

City which has high population or great growth rate might have specific reason. For exsample, some country has great health service or greate economy system. We wanted to make sure these expectation, so we've chosen these 2 dataset. Because'2023 Global Country Development & Prosperity Index' has score from many point of views, it it going to be useful to analyze.

## Data analysis

### Save data into PostgreSQL

Before running following code, we made database 'final_project' through PGadmin beforehand.
To connect local postgreSQL, run this code

In [51]:
import psycopg2
# Connect to your postgres DB
passward = input("Enter your password: ")

conn = psycopg2.connect(
dbname="final_project",
user="postgres",
password=passward,
host="localhost",
port="5432"
)
cur = conn.cursor()

### Create table and import data from csv

In [2]:
# Create a new table
cur.execute(
    "CREATE TABLE population_growth ("
    "id SERIAL PRIMARY KEY, "
    "city varchar, "
    "country varchar, "
    "continent varchar, "
    "population_2024 integer, "
    "population_2023 integer, "
    "growth_rate decimal)"
)
# Commit the changes
conn.commit()

In [3]:
import csv

with open('./Wprld population growth rate by cities 2024.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        values = [str(d) for d in row[:6]]
        values[0] = "'" + values[0] + "'"
        values[1] = "'" + values[1] + "'"
        values[2] = "'" + values[2] + "'"
        
        values = ", ".join(values)

        cur.execute(
            "INSERT INTO population_growth "
            "(city, country, continent, population_2024, population_2023, growth_rate) "
            "VALUES ("+values+")"
        )
    conn.commit()


In [4]:
# Create a new table
cur.execute(
    "CREATE TABLE global_country_development ("
    "id SERIAL PRIMARY KEY, "
    "country varchar, "
    "average_score decimal, "
    "safety_security decimal, "
    "personel_freedom decimal, "
    "governance decimal, "
    "social_capital decimal, "
    "investment_enviroment decimal, "
    "enterprise_conditions decimal, "
    "market_access_infrastructure decimal, "
    "economic_quality decimal, "
    "living_conditions decimal, "
    "health decimal, "
    "education decimal, "
    "national_enviroment decimal)"
)
# Commit the changes
conn.commit()

In [5]:
with open('./2023 Global Country Development & Prosperity Index.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        values = [str(d) for d in row[:14]]
        values[0] = "'" + values[0] + "'"
        values = ", ".join(values)
        
        cur.execute(
            "INSERT INTO global_country_development "
            "(country, average_score, safety_security, personel_freedom, governance, social_capital, investment_enviroment, enterprise_conditions, "
            "market_access_infrastructure, economic_quality,living_conditions, health, education, national_enviroment) "
            "VALUES ("+values+")"
        )
        
    conn.commit()

### Create temporary table

- population_growth table has approximately 800 cities, but we focus on just one city for each country. we chose city which has highest population in 2024.
- After making subquery, we use "JOIN" statement to combine global_country_development table.

In [52]:
cur.execute(
    "WITH subquery_hightest_population_cities AS ("
        "SELECT city, country, continent, population_2024, growth_rate "
        "FROM ("
            "population_growth "
            "JOIN ("
                "SELECT MAX(population_2024) AS max_population_2024 "
                "FROM population_growth "
                "GROUP BY country "
            ") ON population_2024 = max_population_2024"
        ") "
    ") "

    "SELECT * "
    "INTO TEMP TABLE temp_hightest_population_cities_with_development "
    "FROM subquery_hightest_population_cities "
    "JOIN global_country_development USING (country) "
)


Just to check we could store data as we wanted, display 10 data.

In [55]:
cur.execute(
    "SELECT * "
    "FROM temp_hightest_population_cities_with_development "
    "LIMIT 10"
)

cur.fetchall()

[('Japan',
  'Tokyo',
  'Asia',
  37115035,
  Decimal('-0.0021'),
  16,
  Decimal('78.22'),
  Decimal('92.78'),
  Decimal('79.14'),
  Decimal('79.67'),
  Decimal('43.82'),
  Decimal('83.1'),
  Decimal('80.11'),
  Decimal('79.32'),
  Decimal('66.35'),
  Decimal('92.86'),
  Decimal('86.5'),
  Decimal('84.93'),
  Decimal('70.11')),
 ('India',
  'Delhi',
  'Asia',
  33807403,
  Decimal('0.0263'),
  102,
  Decimal('53.66'),
  Decimal('51.89'),
  Decimal('47.4'),
  Decimal('56.03'),
  Decimal('48.42'),
  Decimal('56.59'),
  Decimal('64.33'),
  Decimal('56.59'),
  Decimal('48.4'),
  Decimal('55.9'),
  Decimal('66.16'),
  Decimal('50.46'),
  Decimal('41.78')),
 ('China',
  'Shanghai',
  'Asia',
  29867918,
  Decimal('0.0225'),
  54,
  Decimal('62.15'),
  Decimal('67.96'),
  Decimal('21.22'),
  Decimal('48.47'),
  Decimal('62.67'),
  Decimal('57.29'),
  Decimal('64.41'),
  Decimal('69.02'),
  Decimal('68.46'),
  Decimal('85.05'),
  Decimal('83.11'),
  Decimal('70.42'),
  Decimal('47.69')),
 ('B

In [50]:
cur.close()
conn.close()